In [ ]:
### EXP2.4: TR -> EN

In [ ]:
### Install Required Packages
!pip install jsonlines sentencepiece # nltk evaluate unbabel-comet accelerate

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
drive_PATH = '../content/drive/MyDrive/Colab Notebooks/l101.experiments.1'

In [ ]:
# General
import numpy as np
import tqdm
import random

# Data
import jsonlines
import torch

# Model
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M") # Load Model
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M") # Load Tokenizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
### Dataset
test_tr_PATH = drive_PATH + '/data/datasets/test_tr_1000.jsonl'
test_de_PATH = drive_PATH + '/data/datasets/test_de_1000.jsonl'
test_tr = []
test_de = []

with jsonlines.open(test_tr_PATH) as f:
    for line in f.iter():
        test_tr.append(line)
# with jsonlines.open(test_de_PATH) as f:
#     for line in f.iter():
#         test_de.append(line)

In [ ]:
# SRC: TR
# TGT: EN
src_lang = "tr"
tgt_lang = "en"

In [ ]:
### Setting the Language Configurations
dataset = test_tr
tokenizer.src_lang = src_lang
print(len(dataset))

In [ ]:
### Decoding: Epsilon Sampling

res = []
temperature = 1
epsilon_cutoff = 0.02
num_candidates = 128

for sent in tqdm.tqdm(dataset):
  #print(sent['en'])
  src_sent = sent[src_lang]
  encoded_src_sent = tokenizer(src_sent, return_tensors="pt").input_ids.to("cuda")

  candidates = []
  for idxc in range(num_candidates):
    tgt_tokens = model.generate(encoded_src_sent, forced_bos_token_id=tokenizer.get_lang_id(tgt_lang), do_sample=True, temperature=temperature, epsilon_cutoff=0.02, num_beams=1, early_stopping=False, num_return_sequences=1)
    tgt_sent = tokenizer.batch_decode(tgt_tokens, skip_special_tokens=True)
    candidates.append(tgt_sent)

  res_sent = {'src': sent[src_lang], 'candidates': candidates, 'ref':sent[tgt_lang]}
  res.append(res_sent)

cand_de_PATH = drive_PATH+ f'/exp2.4.tren-n{num_candidates}-t{temperature}-e{epsilon_cutoff}.jsonl'
with jsonlines.open(cand_de_PATH, mode='w') as writer:
    for item in res:
        writer.write(item)

In [ ]:
res